# Baseline Model for cs231n/bmi260 Project

Housekeeping / Preprocessing the data 

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from skimage import transform
import pandas as pd
import torchvision.datasets as dset
import torchvision.transforms as T
import pydicom
import os
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from tqdm import tnrange, tqdm_notebook

In [32]:

#Change to where your dataset is stored
scans_path = "../medgift.hevs.ch/ILD_DB_lungMasks"

list_of_scans = os.listdir(scans_path)
# for figuring out the controls lets experiment with slice 122 of slice 2
scan_num = 1
scan_path = os.path.join(scans_path,list_of_scans[scan_num])
list_of_slices = os.listdir(scan_path)
slice_num = 10
slice_path = os.path.join(scan_path,list_of_slices[slice_num])

# read in the full path to the file as ds
ds=pydicom.read_file(slice_path) # you may have to use pydicom instead of dicom 

In [5]:
rawimg= ds.pixel_array
plt.imshow(rawimg, cmap='viridis')
plt.show()
print(type(rawimg), np.mean(ds.pixel_array), rawimg.dtype)

<class 'numpy.ndarray'> 425.625228882 uint16


In [6]:
num_scans = len(list_of_scans)
max_num_slices = 0

for scan_num in range(num_scans):
    scan_path = os.path.join(scans_path,list_of_scans[scan_num])
    if (not os.path.isdir(scan_path)) or (list_of_scans[scan_num] == "HRCT_pilot"):
        continue
    list_of_slices = os.listdir(scan_path)
    num_slices = len(list_of_slices)
    num_slices = len(list_of_slices)
    if (num_slices > max_num_slices):
        max_num_slices = num_slices

In [23]:
#maximum number of slices per scan 
max_num_slices = 62
print(list_of_scans, len(list_of_scans))

X = np.zeros((num_scans-2, 512, 512, max_num_slices))
counter = 0;
#produce X array 
for scan_num in tnrange(num_scans):
    scan_path = os.path.join(scans_path,list_of_scans[scan_num])
    if (not os.path.isdir(scan_path)) or (list_of_scans[scan_num] == "HRCT_pilot"):
        #print('bad', list_of_scans[scan_num])
        continue
    list_of_slices = os.listdir(scan_path)
    num_slices = len(list_of_slices)
    scan_idx = int(list_of_scans[scan_num])
    
    for slice_num in range(num_slices):
        if (list_of_slices[slice_num][-4:] != ".dcm"):
            #print(list_of_slices[slice_num])
            continue
            
        slice_path = os.path.join(scan_path,list_of_slices[slice_num])
        ds=pydicom.read_file(slice_path)
        #print(slice_path, ds.RescaleSlope, ds.RescaleIntercept)
        hu_img = ds.pixel_array*ds.RescaleSlope + ds.RescaleIntercept
        if(hu_img.shape != (512,512)):
            hu_img = transform.resize(hu_img, (512,512), mode='constant')
            
        slice_idx_num = int(ds.InstanceNumber)
        X[counter, :, :, slice_idx_num] = hu_img
    counter += 1
    
print(X.shape)
np.save("X", X)

['135', '132', '157', '150', '159', '166', '105', '134', '160', '158', '167', '142', '129', '116', '118', '127', '120', '143', '144', '121', '119', '126', '128', '153', '154', '162', '165', '131', '136', '109', '107', '138', '164', '163', '155', '152', '101', '137', '108', '130', '112', '123', '124', '12', '122', '147', '140', '.listing', '149'] 49
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
bad 167
CT-INSPIRIUM-2951
.listing
CT-INSPIRIUM-2950
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
CT-INSPIRIUM-6409
CT-INSPIRIUM-6410
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing
lung_mask
.listing


In [15]:
df=np.asarray(pd.read_csv('labels.csv', sep=',',header=None).values)
y = np.zeros((X.shape[0], 1))

counter = 0
for scan_num in tnrange(num_scans):
    
    #get rid of other entries in dir 
    scan_path = os.path.join(scans_path,list_of_scans[scan_num])
    if (not os.path.isdir(scan_path)) or (list_of_scans[scan_num] == "HRCT_pilot"):
        continue
    
    idx = np.where(df == int(list_of_scans[scan_num]))[0][0]
    y[counter] = df[idx, 1]
    counter+=1
np.save("y", y)

In [18]:
print(X.shape, y.shape)

(47, 512, 512, 62) (47, 1)
